# Notebook to Continue Scraping

---

This current notebook continues scraping from where it was previously left off. It is meant to be run after the initial scraping notebook, `initial-scraper.ipynb`, has been run.

In [1]:
import os
import requests
import json
import time
import calendar
import numpy as np

from datetime import date, timedelta
from pathlib import Path

In [41]:
def get_last_available_date():
    '''
    Function to automatically obtain the last available date of the data and return it
        @param None: No parameters, as this automatically obtains the last available date
        @return last_date: The last available date of the data
    '''
    max_year = max([int(year_folder) for year_folder in os.listdir('./full-dict')]) ## Get last year
    max_month = max([int(month_folder) for month_folder in os.listdir(f'./full-dict/{max_year}')]) ## Get last month
    last_date = [x[:-5] for x in os.listdir(f'./full-dict/{max_year}/{max_month}/')][-1] ## get last date from the file

    year, month, day = map(int, last_date.split('-')) # Assuming the date format is YYYY-MM-DD, split the string to get year, month, and day

    return date(year, month, day) # Return a datetime.date object

In [ ]:
def update_return_dict(articles, cur_date):
    '''
    This function processes all of the articles associated with a specific date, it checks:
        1) Whether the current DOI has ALREADY been processed, if so updates the saved JATSXML link in associated JSON
        2) Whether the key for the license associated with the current article exists, if so, a new key is then created and added to the JSON value for that key in particular
            @param article: The article JSON object to be processed
            @return: None
    '''
    cur_date = str(cur_date)
    articles = articles['collection']
    for article in articles:
        ## Tracks the number of license type count from the return_dict "license" object
        return_dict['license'][article['license']] = return_dict['license'].get(article['license'], 0) + 1

        ## Adds/UPDATES the most recent DOI submission from return_dict
        return_dict['content'][article['doi']] = {
            'date': article['date'],
            'license': article['license'],
            'jatsxml': article['jatsxml']
        }

In [ ]:
def update_full_dict_by_date(articles, cur_date):
    '''
    This function adds to a dict object where the keys are the dates in string format and the values are a list of DOIs for that date
        @param articles: All articles in current date to extract information from
        @param cur_date: The current date to process in YYYY-mm-dd format
        @return: None
    '''
    cur_date = str(cur_date)
    articles = articles['collection']
    
    full_dict_by_date[cur_date] = [{'doi': item['doi'], 'license': item['license']} for item in articles]

In [ ]:
def get_dated_subfolders(date):
    '''
    Function to create subfolders for each of the day, month, year in the current date, specifically to store metadata from "full_dict_by_date"
        @param date: The current date in YYYY-mm-dd format
        @return: The full output path for the folder representing the current date
    '''
    month = date.month
    year = date.year

    dict_main_folder = Path('./full-dict')

    dict_year_folder = dict_main_folder.joinpath(str(year))
    dict_year_folder.mkdir(exist_ok=True)

    dict_month_folder = dict_year_folder.joinpath(str(month))
    dict_month_folder.mkdir(exist_ok=True)

    return dict_month_folder

In [ ]:
def split_dates(dates, num_splits):
    '''
    Function that takes in a list of dates (or anything, really) and splits those dates into num_splits number of splits
        @param dates: A list of dates (or anything really)
        @param num_splits: The number of sub-lists to split the original list into
    '''
    first_last_dates_fx = lambda lis: [[li[0], li[-1]] for li in lis] ## Essentially extracts the first and last elements of the split list of dates
    return list(first_last_dates_fx(np.array_split(dates, num_splits))) ## Returns the first and last dates of each split

In [ ]:
def get_split_folder(split_num):
    '''
    Function that creates a folder for the specific split applicable to the current operation
        @param split_num: The split number to obtain the associated folder for
        @return: The path to the folder for the split number
    '''
    split_main_folder = Path('./amend_dicts/')
    split_main_folder.mkdir(exist_ok=True)

    split_folder = split_main_folder.joinpath(str(split_num))
    split_folder.mkdir(exist_ok=True)

    return split_folder ## Not needed but just in case

In [ ]:
def process_bioRxiv(start_date=lambda _: date(2013, 1, 1),
                    end_date=lambda _: date.today(),
                    load_dicts=False,
                    delay=5,
                    **kwargs):
    '''
    Function to process the data (loading in if necessary) for a period of time defined by:
        @param start: Beginning date in the format YYYY-mm-dd
        @param end: End date in the format YYY-mm-dd
        @param save_every: Saves (overrites) the last dict every n number of iterations through the data
        @param load_dicts: Loads in the dictionary (JSON) objects associated with "full_dict_by_date" and "return_dict"
    '''
    global full_dict_by_date
    full_dict_by_date = {}

    global return_dict
    return_dict = {
        'license': {},
        'content': {}
    }

    get_split_folder(SELECTED_NUMBER) ## Creates the folder for the current split number

    if load_dicts:
        with open(f'{SELECTED_NUMBER if SELECTED_NUMBER else "."}/full_dict_by_date.json', 'r') as f:
            full_dict_by_date = json.load(f)
        with open(f'{SELECTED_NUMBER if SELECTED_NUMBER else "."}/return_dict.json', 'r') as f:
            return_dict = json.load(f)

        start_date = date(full_dict_by_date.keys().sort()[-1]) + timedelta(days=1) ## Gets the last available date and adds 1 IF not the first date, otherwise starts at Jan 1, 2013

    dates = [start_date + timedelta(days=x) for x in range((end_date  - start_date).days + 1)] ## Gets a list of available dates

    for cur_date in dates:

        flag = True ## Flag of whether or not to continue the loop to get more pages
        num_skip = 0 ## Flag to skip as per API instructions
        while flag:
            cur_date_dir = get_dated_subfolders(cur_date)

            url = f'https://api.biorxiv.org/details/biorxiv/{str(cur_date)}/{str(cur_date)}/{num_skip}'

            response = requests.get(url) ## Processes request and waits for response / articles in JSON format
            response.raise_for_status()
            articles = response.json()

            if articles['messages'][0]['status'] != 'ok': ## Continues onto the next date if posts for the current date are unavailable
                flag = False
                continue

            update_full_dict_by_date(articles=articles, cur_date=cur_date)
            update_return_dict(articles=articles, cur_date=cur_date)

            print(f'Saving "full_dict_by_date.json" to folder "{SELECTED_NUMBER}"...')
            with open(f'./output_dicts/{SELECTED_NUMBER}/full_dict_by_date.json', 'w') as f:
                json.dump(full_dict_by_date, f, indent=2)
            
            print(f'Saving "return_dict.json" to folder "{SELECTED_NUMBER}"...')
            with open(f'./output_dicts/{SELECTED_NUMBER}/return_dict.json', 'w') as f:
                json.dump(return_dict, f, indent=2)

            print(f'Saving "full_dict_by_date.json" to folder "{str(cur_date_dir)}"...') ## Saves the updated JSONs
            with open(f'{str(cur_date_dir)}/{cur_date}.json', 'w') as f: ## Saves the whole output as today's date
                json.dump(articles['collection'], f, indent=2)

            print('='*50)
            print('Processed date:', cur_date, 'with', len(articles['collection']), 'articles')
            print(f'Sleeping for {delay} second(s)... Zzz...')
            print('='*50)
            time.sleep(delay) ## Uses the default 5 seconds if need be

            num_skip += 100

In [ ]:
global DO_NOT_CHANGE_FLAG
DO_NOT_CHANGE_FLAG = 10 ## Do not change this number, it is the number of splits to create and is set standard to 100

In [ ]:
LEFT_OFF = 0 ## The split number to start from, if 0, then starts from the beginning

In [ ]:
USE_TODAY = True ## Whether or not to use today's date as the end date or to use the end date specified below

# Modified API Call Function

---

The below scraper function utilizes a modified start parameter in order to begin from where the last available date for data was scraped.

In [ ]:
for num in range(LEFT_OFF, DO_NOT_CHANGE_FLAG):
    global SELECTED_NUMBER
    SELECTED_NUMBER = num ## Enter selected number here, which represents the selected number

    dates = [get_last_available_date() + timedelta(days=x) for x in range((date.today() - get_last_available_date()).days + 1)] ## Gets a list of available dates
    split_dates_list = split_dates(dates, DO_NOT_CHANGE_FLAG) ## Splits the dates into the selected number of splits

    start_date = split_dates_list[SELECTED_NUMBER][0] ## Gets the start date from the selected number
    end_date = split_dates_list[SELECTED_NUMBER][1] ## Gets the end date from the selected number

    process_bioRxiv(start_date=start_date, end_date=end_date, load_dicts=False, delay=1)

# Modified Combine Function

---

The below combine function utilizes a modified start parameter in order to begin from where the last available date for data was scraped

In [ ]:
def combine_output_dicts(path='./output_dicts/', save_path='./global_dicts/', num=0):
    '''
    Function to combine all of the output dicts into one JSON file by creating a placeholder, temporary dict and adding all of the values from the other dicts to it
        This operates on the previously defined global dictionalry objects "full_return_dict" and "full_return_dict_by_date"
        @param path: The path to the folder containing all of the output dicts
        @return: None
    '''
    try:
        with open(f'{path}/{num}/return_dict.json', 'r') as f:
            cur_return_dict = json.load(f)

        with open(f'{path}/{num}/full_dict_by_date.json', 'r') as f:
            cur_return_dict_by_date = json.load(f)

        print(f'Combining and saving "return_dict" license values for folder {num}...')
        global_return_dict['license'] = {key: global_return_dict['license'].get(key, 0) + value for key, value in cur_return_dict['license'].items()} ## Adds the license counts to the global dict for each license count
        global_return_dict['content'].update(cur_return_dict['content']) ## Adds the content to the global dict by date
        with open(f'{save_path}/return_dict.json', 'w') as f:
            json.dump(global_return_dict, f, indent=2)

        print(f'Combining and saving "full_dict_by_date" content items for folder {num}...')
        global_return_dict_by_date.update(cur_return_dict_by_date) ## Adds the content to the global return dict by date
        with open(f'{save_path}/full_dict_by_date.json', 'w') as f:
            json.dump(global_return_dict_by_date, f, indent=2)
        
    except FileNotFoundError:
        print(f'No files in folder {num}, continuing to the next folder...')

In [ ]:
def get_global_folder():
    '''
    Function to create the global folder for the combined JSONs
        @return: The path to the global folder
    '''
    global_folder = Path('./global_dicts/')
    global_folder.mkdir(exist_ok=True)

    return str(global_folder)

In [ ]:
global global_return_dict
with open('./global_dicts/return_dict.json', 'r') as f:
    global_return_dict = json.load(f)

global global_return_dict_by_date
with open('./global_dicts/return_dict_by_date.json', 'r') as f:
    global_return_dict_by_date = json.load(f)

In [ ]:
for num in range(DO_NOT_CHANGE_FLAG):
    combine_output_dicts(path='./amend_dicts/',save_path=lambda _: get_global_folder(), num=num)

# Sort By License

---

Below we finally sort the `return_dict` based on license type and output individual `.JSON` files.